In [13]:
from codebook import Codebook

vq = Codebook(
    input_size=8,
    embedding_size=8,
    num_embedding=50,
    use_cosine_sim=True
)

import torch
T = 10
x = torch.randn(2, T, 8)

In [14]:
code = vq.quantize_to_code(x)

code.shape, code

(torch.Size([2, 10]),
 tensor([[ 3, 22,  0, 24, 27, 33, 20,  0, 10, 13],
         [ 2, 20, 25,  9,  2, 15,  8, 34, 22, 37]]))

In [28]:
one_hot = torch.nn.functional.one_hot(code, num_classes=1024)
counts = one_hot.sum(dim=1)
counts / T

tensor([[0.2000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2000,  ..., 0.0000, 0.0000, 0.0000]])

In [17]:
dist / (T*2)

tensor([0.1000, 0.0000, 0.1000, 0.0500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0500,
        0.0500, 0.0500, 0.0000, 0.0000, 0.0500, 0.0000, 0.0500, 0.0000, 0.0000,
        0.0000, 0.0000, 0.1000, 0.0000, 0.1000, 0.0000, 0.0500, 0.0500, 0.0000,
        0.0500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0500, 0.0500, 0.0000,
        0.0000, 0.0500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [22]:
dec = torch.randn(4, 2, 2)
residual_q = torch.ones(4,2,2)
mask = (
                torch.full((dec.shape[0],), fill_value=False, device=dec.device)
            )
residual_q *= mask[:, None, None]
dec

tensor([[[ 0.0040,  0.9420],
         [ 0.5901, -1.5170]],

        [[-1.7792, -0.1886],
         [-0.5103,  0.1227]],

        [[-1.1521, -0.6097],
         [ 0.8428,  1.3517]],

        [[-1.0177,  1.8646],
         [ 2.5531, -0.0660]]])

In [23]:
residual_q

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [24]:
torch.nn.Identity()(residual_q + dec)

tensor([[[ 0.0040,  0.9420],
         [ 0.5901, -1.5170]],

        [[-1.7792, -0.1886],
         [-0.5103,  0.1227]],

        [[-1.1521, -0.6097],
         [ 0.8428,  1.3517]],

        [[-1.0177,  1.8646],
         [ 2.5531, -0.0660]]])

In [1]:
from finite_scalar_quantize import FSQ
fsq = FSQ(levels=[2]*2, num_codebooks=1, dim=6, )

In [2]:
fsq.training

True

In [3]:
fsq.eval()
fsq.training

False

In [4]:
import torch
x = torch.randn(1, 2, 6)
x

tensor([[[-0.2831,  0.1810,  1.0986, -1.2818,  0.7840, -1.2412],
         [ 0.0375, -0.7446,  0.9225, -1.0288,  0.2784, -0.0822]]])

In [5]:
out, indices = fsq(x)
print(out.shape, indices.shape)
out, indices

torch.Size([1, 2, 6]) torch.Size([1, 2])


(tensor([[[ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975],
          [ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975]]],
        grad_fn=<ViewBackward0>),
 tensor([[3, 3]], dtype=torch.int32))

In [6]:
fsq.train()
print(fsq.training)

out, indices = fsq(x)
print(out.shape, indices.shape)
out, indices

True
torch.Size([1, 2, 6]) torch.Size([1, 2])


(tensor([[[ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975],
          [ 0.4075,  0.2481,  0.1417,  0.4198, -0.5422, -0.1975]]],
        grad_fn=<ViewBackward0>),
 tensor([[3, 3]], dtype=torch.int32))